In [30]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm_notebook

In [31]:
df = pd.read_csv('data/train.csv')
df = df.rename({'Company ': 'Company'}, axis=1)
df = df.sort_values(['Company', 'ID'])
df = df.reset_index()

features = df.columns[4:-1].tolist()
next_records_count = 10

for feature in features:
    for i in range(1, next_records_count + 1):
        df.insert(df.shape[1], '{0}_{1}'.format(feature, i), np.nan)

for index, row in tqdm_notebook(df.iterrows(), total=df.shape[0], desc='Filling additional features'):
    for feature in features:
        for i in range(1, next_records_count + 1):
            if index + i < df.shape[0] and df.at[index + i, 'Company'] == row['Company']:
                df.at[index, '{0}_{1}'.format(feature, i)] = df.loc[index + i, feature]

df = df.dropna()
df[:5]

,index,ID,Date,Company,SMA,EMA,WMA,DEMA,TEMA,TRIMA,...,QUADRATURE_1,QUADRATURE_2,QUADRATURE_3,QUADRATURE_4,QUADRATURE_5,QUADRATURE_6,QUADRATURE_7,QUADRATURE_8,QUADRATURE_9,QUADRATURE_10
54,162,163,03-04-2148,ABC,26.936,26.9160,27.1042,27.5128,27.2546,26.9737,...,0.1485,0.5227,0.0459,-1.1171,-1.6814,-2.1545,-2.1983,-0.4979,1.9877,4.4819
55,165,166,06-04-2148,ABC,27.042,26.9313,27.1158,27.4320,27.1422,27.0740,...,0.5227,0.0459,-1.1171,-1.6814,-2.1545,-2.1983,-0.4979,1.9877,4.4819,5.0483
56,168,169,07-04-2148,ABC,26.986,26.7274,26.8918,26.9703,26.5223,27.0930,...,0.0459,-1.1171,-1.6814,-2.1545,-2.1983,-0.4979,1.9877,4.4819,5.0483,4.1573
57,171,172,08-04-2148,ABC,26.749,26.4588,26.5762,26.4377,25.8552,27.0033,...,-1.1171,-1.6814,-2.1545,-2.1983,-0.4979,1.9877,4.4819,5.0483,4.1573,2.0008
58,174,175,09-04-2148,ABC,26.580,26.2045,26.2691,25.9792,25.3354,26.8447,...,-1.6814,-2.1545,-2.1983,-0.4979,1.9877,4.4819,5.0483,4.1573,2.0008,-2.6105


In [32]:
corr = df[df.columns[3:]].corr().sort_values('Price', ascending=False)['Price']
corr

Price                 1.000000
TEMA_2                0.999731
DEMA_3                0.999658
TRIMA_6               0.999640
DEMA_2                0.999600
TEMA_3                0.999586
WMA_4                 0.999575
TRIMA_5               0.999555
TRIMA_7               0.999536
WMA_5                 0.999510
WMA_3                 0.999495
Real Middle Band_6    0.999481
SMA_6                 0.999481
DEMA_4                0.999477
TEMA_1                0.999442
SMA_7                 0.999428
Real Middle Band_7    0.999428
SMA_5                 0.999425
Real Middle Band_5    0.999425
T3_7                  0.999417
T3_8                  0.999395
EMA_4                 0.999380
EMA_5                 0.999371
T3_6                  0.999315
WMA_6                 0.999315
TRIMA_4               0.999295
EMA_3                 0.999276
TRIMA_8               0.999264
EMA_6                 0.999263
SMA_4                 0.999256
                        ...   
MACD_Hist_8          -0.050569
Aroon Do

In [41]:
model_vars = [name for name in corr.index if corr[name] < 1.0 and corr[name] > 0.5]
model = sm.OLS(df['Price'], df[model_vars]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.152e+06
Date:                Thu, 04 Jul 2019   Prob (F-statistic):               0.00
Time:                        14:31:35   Log-Likelihood:                -18971.
No. Observations:               11805   AIC:                         3.827e+04
Df Residuals:                   11643   BIC:                         3.946e+04
Df Model:                         162                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
TEMA_2                135.6199    171.891      0.789      0.430    -201.315     472.555
DEMA_3                 26.7007     27.759      0.962      0.336     -27.711      81.112
TRIMA_6              -391.6689    408.254     -0.959      0.337   -1191.916     408.578
DEMA_2                  7.5997     22.108      0.344      0.731     -35.735      50.934
TEMA_3                 39.8585    174.977      0.228      0.820    -303.125     382.842
WMA_4                -161.4772    377.092     -0.428      0.669    -900.641     577.687
TRIMA_5               360.8939    400.033      0.902      0.367    -423.238    1145.026
TRIMA_7              -325.4211    411.836     -0.790      0.429   -1132.689     481.847
WMA_5                 379.9038    378.128      1.005      0.315    -361.291    1121.099
WMA_3                -403.0032    376.389     -1.071      0.284   -1140.789     334.783
Real Middle Band_6    145.2041    173.359      0.838      0.402    -194.609     485.017
SMA_6                 144.8427    173.507      0.835      0.404    -195.261     484.946
DEMA_4                 29.3945     26.851      1.095      0.274     -23.237      82.026
TEMA_1                -60.3912    161.001     -0.375      0.708    -375.981     255.198
SMA_7                 139.1066    191.154      0.728      0.467    -235.587     513.801
Real Middle Band_7    138.6616    192.069      0.722      0.470    -237.825     515.148
SMA_5                 125.2576    201.069      0.623      0.533    -268.872     519.387
Real Middle Band_5    125.4977    200.946      0.625      0.532    -268.391     519.387
T3_7                   35.2377    355.397      0.099      0.921    -661.401     731.876
T3_8                  255.3650    349.003      0.732      0.464    -428.738     939.469
EMA_4                 -61.8297    356.451     -0.173      0.862    -760.533     636.873
EMA_5                -683.3019    360.249     -1.897      0.058   -1389.450      22.847
T3_6                   -8.3699    358.054     -0.023      0.981    -710.216     693.476
WMA_6                 329.5654    376.967      0.874      0.382    -409.353    1068.484
TRIMA_4              -878.1314    401.786     -2.186      0.029   -1665.700     -90.563
EMA_3                 612.3481    349.176      1.754      0.080     -72.096    1296.792
TRIMA_8               425.5483    411.225      1.035      0.301    -380.522    1231.618
EMA_6                -352.7108    363.696     -0.970      0.332   -1065.617     360.195
SMA_4                  79.7239    146.586      0.544      0.587    -207.608     367.056
Real Middle Band_4     79.7255    146.585      0.544      0.587    -207.606     367.057
T3_9                  175.9618    338.640      0.520      0.603    -487.830     839.754
Real Middle Band_8    -37.3175    277.834     -0.134      0.893    -581.919     507.284
SMA

In [42]:
full_names = ['simpleMovingAverage', 'exponentialMovingAverage', 'waysAndMeansAdvances', 'doubleExponentialMovingAverage', 'tripleExponentialMovingAverage', 'triangularMovingAverage', 'kaufmansAdaptiveMovingAverage', 'fAdaptiveMovingAverage', 'mesaAdaptiveMovingAverage', 't3MovingAverage', 'movingAverageConvergenceDivergence', 'movingAverageConvergenceDivergenceHistogram', 'movingAverageConvergenceDivergenceSignal', 'movingAverageConvergence', 'movingAverageConvergenceHistogram', 'movingAverageConvergenceSignal', 'slowStochasticOscillatorSecondLine', 'slowStochasticOscillatorMainLine', 'fastStochasticOscillatorSecondLine', 'fastStochasticOscillatorMainLine', 'relativeStrengthIndex', 'fastStochasticSecondLine', 'fastStochasticMainLine', 'willr', 'averageDirectionalIndex', 'averageDirectionalIndexRating', 'absolutePriceOscillator', 'percentagePriceOscillator', 'momentumIndicator', 'balanceOfPower', 'commodityChannelIndex', 'chandeMomentumOscillator', 'rateOfChange', 'rateOfChangeRating', 'aroonOscillatorDown', 'aroonOscillatorUp', 'aroonOscillator', 'moneyFlowIndex', 'tripleSmoothedExponentialMovingAverage', 'ultimateOscillatorDefinitionAndStrategies', 'directionalMovementIndex', 'minusDirectionalIndex', 'plusDirectionalIndex', 'minusDirectionalMovementIndex', 'plusDirectionalMovementIndex', 'relativeStrengthIndexLowerBand', 'relativeStrengthIndexMiddleBand', 'relativeStrengthIndexHighBand', 'midPoint', 'midPrice', 'parabolicStopAndReverse', 'trange', 'averageTrueRange', 'normalizedAverageTrueRange', 'chaikinAccumulationDistributionLine', 'accumulationDistributionOscillatorSC', 'onBalanceVolume', 'htTrendline', 'leadSine', 'sine', 'trendMode', 'dcPeriod', 'htDcPhase', 'phase', 'quadrature']

indexes = [int(name.rsplit('_', 1)[1]) for name in model.params.index if '_' in name and name not in features]

if len(indexes) > 0:
    print('if (nextRecords.size < {0}) {{'.format(max(indexes)))
    print('    return record.tripleExponentialMovingAverage')
    print('}')
    print()

print('return listOf(')
    
for name in model.params.index:
    modifier = model.params[name]
    comma = ',' if name != model.params.index[-1] else ''
    
    if '_' in name and name not in features:
        parts = name.rsplit('_', 1)
        index = int(parts[1]) - 1
        full_name = full_names[features.index(parts[0])]
        
        print('    nextRecords[{0}].{1} * {2}{3}'.format(index, full_name, modifier, comma))
    else:
        full_name = full_names[features.index(name)]
        
        print('    record.{0} * {1}{2}'.format(full_name, modifier, comma))

print(').sum()')

if (nextRecords.size < 10) {
    return record.tripleExponentialMovingAverage
}

return listOf(
    nextRecords[1].tripleExponentialMovingAverage * 135.61993554112087,
    nextRecords[2].doubleExponentialMovingAverage * 26.700675373049194,
    nextRecords[5].triangularMovingAverage * -391.6689489096425,
    nextRecords[1].doubleExponentialMovingAverage * 7.599700023506784,
    nextRecords[2].tripleExponentialMovingAverage * 39.85854320528597,
    nextRecords[3].waysAndMeansAdvances * -161.4772008373675,
    nextRecords[4].triangularMovingAverage * 360.89390554488773,
    nextRecords[6].triangularMovingAverage * -325.4211113492365,
    nextRecords[4].waysAndMeansAdvances * 379.9038193348897,
    nextRecords[2].waysAndMeansAdvances * -403.00321530980364,
    nextRecords[5].relativeStrengthIndexMiddleBand * 145.20407218969558,
    nextRecords[5].simpleMovingAverage * 144.84268979370972,
    nextRecords[3].doubleExponentialMovingAverage * 29.394450244326435,
    nextRecords[0].tripleExpone